In [1]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
import math
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class DendriticGatedLayer(nn.Module):
    def __init__(self, n_units, n_prev_layer_units, n_network_inputs):
        self.weights = nn.Parameter(torch.zeros(n_units, n_prev_layer_units), requires_grad=True)

class DendriticGatedNet(nn.Module):
    def __init__(self, n_inputs, n_hiddens, n_out, n_layers):
        '''
            To-do
        '''
        
        None
        
    